<a href="https://colab.research.google.com/github/VinhRP/Makeup_Detection_Yolov8/blob/main/Stream_lit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/166.8 GB disk)


In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
import torch
import torchvision
from torchvision import transforms
from typing import List, Tuple
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

res50 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
res50 = resnet50(weights="DEFAULT")

for module, param in zip(res50.modules(), res50.parameters()):
	if isinstance(module, torch.nn.BatchNorm2d):
		param.requires_grad = False

# define the network head and attach it to the model
numFeatures = res50.fc.in_features

headModel = torch.nn.Sequential(
	torch.nn.Linear(numFeatures, 512),
	torch.nn.ReLU(),
	torch.nn.Dropout(0.25),
	torch.nn.Linear(512, 256),
	torch.nn.ReLU(),
	torch.nn.Dropout(0.5),
	torch.nn.Linear(256, 2)
)
res50.fc = headModel

res50.load_state_dict(torch.load("/content/drive/MyDrive/Test.pth"))

device = "cuda" if torch.cuda.is_available() else "cpu"

image_size: Tuple[int, int] = (224, 224)
class_names = ['Make_up', 'Non_Make_up']

image_transform = transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
res50.to(device)
res50.eval()

image_file = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])
if image_file is not None:
  our_image = Image.open(image_file).convert("RGB")
  yolo = YOLO('/content/drive/MyDrive/face_detection_224.pt')
  results = yolo.predict(our_image) # predict on an image
  for r in results:
    cords = r.boxes.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    img = Image.fromarray(r.plot()[..., :: -1]) ## Original image
    img2 = img.crop(cords) ##Croped image
  with torch.inference_mode():
    transformed_image = image_transform(img2).unsqueeze(dim=0)
    target_image_pred = res50(transformed_image.to(device))
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
  information = f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
  st.text(information)
  st.image(our_image, channels="RGB")

Writing app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.625s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.91.111.237
npx: installed 22 in 1.904s
your url is: https://sad-poets-bow.loca.lt
